In [1]:
import requests
# from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import pickle
import time

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from collections import defaultdict

import pipeline as p

%load_ext autoreload
%autoreload 2

In [2]:
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"

pd.set_eng_float_format(accuracy=1, use_eng_prefix=True)

chromedriver = "/Users/kendra/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url_base = 'https://www.pro-football-reference.com'

team_dict = p.open_pkl('Data/team_dictionary.pkl')

Create a dictionary to track which teams & seasons we've scraped data for

### Iterate over team-season pages, and boxscores within a given season:

In [69]:
driver = webdriver.Chrome(chromedriver, desired_capabilities=caps)

# count = 0
# bxsc_html = defaultdict(list)    # [href] = html -- also tracks whether a bxsc href has been visited
# bxsc_refs = {}                   # [team-year] = list of hrefs for boxscores
# season_html = {}                 # [team-year] = html of team season page
year = 2018
bxsc_html = p.open_pkl(f'Data/BxscHtmls/bxsc_html_{str(year)}.pkl')

for href, html in bxsc_html.items():
    if html == None:
        print('Updating ' + href)
        bxsc_html[href] = p.get_bxsc_data(href, driver)
    else:
        print('Already scraped this boxscore')

Updating /boxscores/201812090rai.htm
Updating /boxscores/201811040rav.htm
Updating /boxscores/201810140dal.htm
Updating /boxscores/201811080pit.htm
Updating /boxscores/201811250buf.htm
Updating /boxscores/201810140nyj.htm
Updating /boxscores/201810210nyj.htm
Already scraped this boxscore
Updating /boxscores/201811040buf.htm
Updating /boxscores/201811110rai.htm
Already scraped this boxscore
Updating /boxscores/201811180clt.htm
Updating /boxscores/201811040nwe.htm
Already scraped this boxscore
Updating /boxscores/201811180atl.htm
Updating /boxscores/201811180chi.htm
Already scraped this boxscore
Already scraped this boxscore
Already scraped this boxscore
Already scraped this boxscore
Updating /boxscores/201812160buf.htm
Updating /boxscores/201812240rai.htm
Updating /boxscores/201812020atl.htm
Already scraped this boxscore
Updating /boxscores/201812300nwe.htm
Updating /boxscores/201812020oti.htm
Updating /boxscores/201811180sdg.htm
Updating /boxscores/201810210kan.htm
Updating /boxscores/

Already scraped this boxscore
Updating /boxscores/201811040nor.htm
Updating /boxscores/201810280cin.htm
Updating /boxscores/201811250min.htm
Updating /boxscores/201811250car.htm
Already scraped this boxscore
Already scraped this boxscore
Already scraped this boxscore
Updating /boxscores/201812230mia.htm
Updating /boxscores/201812090mia.htm
Already scraped this boxscore
Already scraped this boxscore
Updating /boxscores/201812090sfo.htm
Updating /boxscores/201810140rai.htm
Updating /boxscores/201810140cle.htm
Updating /boxscores/201812160nyg.htm
Already scraped this boxscore
Updating /boxscores/201811220nor.htm
Already scraped this boxscore
Updating /boxscores/201812020det.htm
Updating /boxscores/201811040was.htm
Updating /boxscores/201811180was.htm
Already scraped this boxscore
Already scraped this boxscore


In [70]:
driver.close()

### Elo Data from FiveThirtyEight

In [71]:
elo_url = 'https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv'
elo_df = pd.read_csv(elo_url)

In [72]:
elo_df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2
0,1920-09-26,1920,0,NaN,RII,STP,1.5k,1.3k,824.7m,175.3m,1.5k,1.3k,48.0,0.0
1,1920-10-03,1920,0,NaN,BFF,WBU,1.5k,1.3k,802.0m,198.0m,1.5k,1.3k,32.0,6.0
2,1920-10-03,1920,0,NaN,CBD,PTQ,1.5k,1.3k,825.3m,174.7m,1.5k,1.3k,48.0,0.0
3,1920-10-03,1920,0,NaN,CHI,MUT,1.4k,1.3k,683.0m,317.0m,1.4k,1.3k,20.0,0.0
4,1920-10-03,1920,0,NaN,RII,MUN,1.5k,1.5k,644.2m,355.8m,1.5k,1.5k,45.0,0.0


In [74]:
import datetime as dt

In [75]:
elo_df['date'] = pd.to_datetime(elo_df['date'], format='%Y-%m-%d')
# Can drop anything before 2007
elo_df = elo_df[elo_df['date'] >= dt.datetime(2007,8,1)].reset_index(drop=True)

p.pkl_this('Data/elo_df.pkl', elo_df)